In [1]:
import torch
import math
import numpy as np
import json

In [2]:
from lyft_simulation import *

In [5]:
num_drivers = 100
num_subgrids = 16

In [12]:
num_subgrids_per_dim = 4
num_subgrids = num_subgrids_per_dim ** 2

In [13]:
subgrid_size = 1.0 / num_subgrids_per_dim
subgrid_bounds = torch.zeros(num_subgrids, 2, 2)
for i in range(num_subgrids_per_dim):
    for j in range(num_subgrids_per_dim):
        subgrid_index = i * num_subgrids_per_dim + j
        subgrid_bounds[subgrid_index, 0] = torch.tensor([i * subgrid_size, (i + 1) * subgrid_size])
        subgrid_bounds[subgrid_index, 1] = torch.tensor([j * subgrid_size, (j + 1) * subgrid_size])


In [15]:
subgrid_bounds.size()

torch.Size([16, 2, 2])

In [28]:
# Assign each driver to a subgrid
driver_subgrids = torch.randint(0, num_subgrids, (num_drivers,))

# Initialize the driver positions randomly within their assigned subgrids
driver_positions = torch.zeros(num_drivers, 2)

In [31]:
for i in range(num_subgrids):
    mask = driver_subgrids == i
    x_min, x_max = subgrid_bounds[i, 0]
    y_min, y_max = subgrid_bounds[i, 1]
    driver_positions[mask, 0] = torch.rand(mask.sum()) * (x_max - x_min) + x_min
    driver_positions[mask, 1] = torch.rand(mask.sum()) * (y_max - y_min) + y_min

In [35]:
# Define the probability distribution for driver idle times
daytime_prob = 0.8  # Probability of drivers working during daytime hours
nighttime_prob = 1 - daytime_prob
daytime_hours = torch.arange(7, 23)  # Daytime hours (7 AM to 10 PM)
nighttime_hours = torch.cat((torch.arange(0, 7), torch.arange(23, 24)))  # Nighttime hours (12 AM to 7 AM and 11 PM to 12 AM)
idle_time_probs = torch.zeros(24)
idle_time_probs[daytime_hours] = daytime_prob
idle_time_probs[nighttime_hours] = nighttime_prob
idle_time_probs /= idle_time_probs.sum()

In [36]:
idle_time_probs

tensor([0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0139])

In [37]:
idle_starttime = torch.multinomial(idle_time_probs, num_drivers, replacement=True) * 60

In [38]:
idle_starttime

tensor([1380,  780,  600,  960,  720,  600,  420,  780, 1080, 1260,  960,  900,
        1080, 1320, 1200, 1260,  840, 1320, 1020, 1380,  420,  900, 1200,  600,
         840,  840, 1260, 1200,  540,  660, 1080, 1200, 1320, 1020,  960, 1380,
         480,  420,  420,  480, 1020,  540,  960, 1320, 1260,  840,  240, 1260,
        1140, 1140, 1260, 1140,  600,  720,  720, 1140,  660, 1260, 1320,  240,
         480, 1020, 1020,  540, 1320,  420, 1200, 1320,  600,  960, 1020,  840,
        1020,  720,  720,  540, 1200, 1320,  960,  540, 1140, 1200, 1080,  720,
         660, 1260,  780,  660,  540,  840,  840,  360,  480,  660,  840,  420,
        1140,    0,  660,  660])

In [24]:
# on avg. a driver can work for 3hrs
mean_idle_time = 180

In [25]:
exponential_dist = torch.distributions.exponential.Exponential(torch.tensor(1.0 / mean_idle_time))

In [26]:
idle_duration = exponential_dist.sample((num_drivers,)).clamp(min=1, max=24 * 60).int().long()

In [27]:
idle_duration

tensor([ 28, 371,   5, 113,  25,  14,  20, 112, 130, 768, 150, 261, 561,  84,
         14,  66,  58, 113,  42,   3,   6,   3, 179, 666,  71, 294,  43,  31,
        127, 113, 517, 198, 123, 122,   7,  30,  34, 143,   4,  87,  11,  14,
         88, 393, 243,  62, 105,  47,  40,   3,  62,  33, 520, 144, 316, 364,
         83,  12, 323,  45, 235,  66, 169,   7, 357, 197, 171, 251,  21,  66,
        346, 398, 671, 130, 105, 508,  93,  14,  14,  83,  31, 112, 333, 185,
        434, 360, 123, 112, 318, 497,  65, 245,  69, 147,  28, 145,  36,  59,
         55, 164])

In [42]:
drivers = torch.cat((idle_starttime.unsqueeze(1), idle_duration.unsqueeze(1), driver_positions), 1)

### test py code

In [3]:
T0_pricing_params = (5, 0.78, 1.82)
lr = 0.01
simulation_week1 = WeeklySimulation(lr, T0_pricing_params)

In [4]:
drivers_week1_S0 = simulation_week1.simulate_drivers()

In [5]:
drivers_week1_S0.size()

torch.Size([100, 4])

In [6]:
drivers_week1_S0[0]

tensor([1.1400e+03, 1.4000e+01, 9.0193e-01, 7.4738e-01])

In [8]:
simulation_week1.get_subblock_index(drivers_week1_S0[:, 2], drivers_week1_S0[:, 3])

tensor([14,  2,  9,  1, 13, 10,  8, 13,  7,  0,  5,  8,  3,  1,  7,  3,  6,  3,
         7,  1,  8,  2,  9, 11,  5,  7, 13, 11,  4,  6,  4,  0,  6,  7, 11,  1,
        12, 12, 14, 15,  0, 14,  3, 13,  9,  1, 13,  4,  0, 13,  1, 12, 15,  6,
         0,  2,  6,  0,  2,  8,  2, 13, 15,  8,  5, 12, 11,  8,  1,  0,  4, 11,
        11,  3, 12,  9, 15,  6,  0,  3,  0,  1,  8,  4,  2,  8,  9,  9, 12,  6,
         6, 11,  3,  3,  9,  2,  9, 12,  0,  7], dtype=torch.int32)

### TODO - 
### since in each request has to be matched to an idle driver within the same sub-block, maybe we don't need to update the driver's location after finishing the trip(to avoid loops).
### the problem is for those trips that cannot be finished within each 30-min interval


### matching - for every 30-min interval, iterate through each sub-block, for rider requests mask on the timestamp and the sub-block id; for drivers, mask on idle(not in busy_drivers and current timestamp within [idle_start_time, idle_start_time+idle_duration]) and sub-block id, randomly pick a valid driver(if no valid driver, cancel the request right away), do the match(acceptance prob. for both parties) and add the current driver and the trip end timestamp to the busy_drivers hashmap.




### if there are so many requests not able to find even a valid driver, may consider add more total drivers.

### how M0 impacts S1

### may still sample idle position and idle time every day and match every 30mins.
### a driver's reject may change the driver's mean_idle_time(for exp var to sample idle time), may or may not affect driver's initial position for the next day